In [77]:
## imports
import pandas as pd
import numpy as np
import mne
import re
import os

from mne.preprocessing import (
    ICA, 
    create_ecg_epochs, 
    corrmap, 
    create_eog_epochs
)

# mne graphics config
# from PySide6 import QtCore, QtGui, QtWidgets, __version__ 

rawa = None
rawb = None

import datetime
print('IMPORTED MODULES today:', datetime.datetime.now())

# Removed cached function import
import sys
try:
    del sys.modules['rename_events']
except KeyError:
    pass

from rename_events import relabel_events


# setup

# SESSION AND POSNER CODES
session_id='m_00_01'
posner_id ='pos1a'
posner_id_b ='pos1b'

IMPORTED MODULES today: 2023-07-17 14:01:23.399032


# Posner A

In [78]:
# Relabel stim events and load raw data
raw_a = relabel_events(session_id, posner_id)

Data file:  m_00_01/m_00_01_pos1a.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '1a' in file: m_00_01/00_1a_posner_task_2023-04-05_15h00.59.401/00_1a_posner_task_2023-04-05_15h00.59.401.csv
Found '1b' in file: m_00_01/00_1b_posner_task_2023-04-05_16h08.54.140/00_1b_posner_task_2023-04-05_16h08.54.140.csv
********** 1a
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Cropping annotations 2023-04-05 15:02:13.096531+00:00 - 2023-04-05 15:20:02.316531+00:00
  [0] Keeping  (2023-04-05 15:02:13.096531+00:00 - 2023-04-05 15:02:13.096531+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-04-05 15:03:13.266531+00:00 - 2023-04-05 1

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [298] Keeping  (2023-04-05 15:12:36.824531+00:00 - 2023-04-05 15:12:36.824531+00:00 -> 623.728 - 623.728)
  [299] Keeping  (2023-04-05 15:12:38.015531+00:00 - 2023-04-05 15:12:38.015531+00:00 -> 624.919 - 624.919)
  [300] Keeping  (2023-04-05 15:12:39.514531+00:00 - 2023-04-05 15:12:39.514531+00:00 -> 626.418 - 626.418)
  [301] Keeping  (2023-04-05 15:12:42.414531+00:00 - 2023-04-05 15:12:42.414531+00:00 -> 629.318 - 629.318)
  [302] Keeping  (2023-04-05 15:12:42.514531+00:00 - 2023-04-05 15:12:42.514531+00:00 -> 629.418 - 629.418)
  [303] Keeping  (2023-04-05 15:12:44.514531+00:00 - 2023-04-05 15:12:44.514531+00:00 -> 631.418 - 631.418)
  [304] Keeping  (2023-04-05 15:12:46.014531+00:00 - 2023-04-05 15:12:46.014531+00:00 -> 632.918 - 632.918)
  [305] Keeping  (2023-04-05 15:12:49.054531+00:00 - 2023-04-05 15:12:49.054531+00:00 -> 635.958 - 635.958)
  [306] Keeping  (2023-04-05 15:12:49.154531+00:00 - 2023-04-05 15:12:49.154531+00:00 -> 636.058 - 636.058)
  [307] Keeping  (2023-04-05

In [79]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events, event_id = mne.events_from_annotations(raw_a, event_map)

print(event_id)
e_a = events
# all rows with fixation (code 40)
e_40_a = e_a[e_a[:, 2] == 40]
# all rows with trial end (code 101)
e_101_a = e_a[e_a[:,2]==101]

a_tmin= e_40_a[0][0]/1000
a_tmax= e_101_a[-1][0]/1000

# Crop and make a copy and filter
filt_raw_a = raw_a.crop(tmin=a_tmin, tmax=a_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_a = mne.Epochs(filt_raw_a, 
                    e_a, 
                    event_id={'fix': 40},
                    tmin=0, 
                    tmax=6,
                    baseline=(0,1),
                    preload=True)

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
{'10': 10, '101': 101, '109': 109, '117': 117, '20': 20, '30': 30, '40': 40, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
100 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 100 events and 6001 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


In [76]:
epochs_a.plot(events=events)

Using pyopengl with version 3.1.6


/var/folders/2b/4tnqzv_x3x17pf4q1_n7m8h40000gn/T/ipykernel_48764/581328209.py:1: RuntimeWarning: More events than default colors available. You should pass a list of unique colors.
  epochs_a.plot(events=events)


Dropped 4 epochs: 38, 41, 45, 82
The following epochs were marked as bad and are dropped:
[156, 168, 184, 338]
Channels marked as bad:
none


In [50]:
raw_a.plot()

Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [80]:
# Fitting ICA
ica = ICA(n_components=len(filt_raw_a.ch_names)-1, max_iter="auto", random_state=999)
ica.fit(filt_raw_a)
ica

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components
Fitting ICA took 48.5s.


Method,fastica
Fit,75 iterations on raw data (685732 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [82]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_a.load_data()
ica.plot_sources(raw_a)

Creating RawArray with float64 data, n_channels=63, n_times=685732
    Range : 131125 ... 816856 =    131.125 ...   816.856 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
342 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
342 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
342 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
342 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
342 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
342 matching events found
No baseline correction applied
0 projection items ac

In [83]:
print(ica.exclude)

[0, 1, 2, 3, 4, 5, 7, 9, 11, 16, 18, 24, 25]


In [84]:
reconstr_raw = raw_a.copy()
# Apply ica to copy
ica.apply(reconstr_raw)
# Plot
raw_a.plot(title="Raw data")
reconstr_raw.plot(title="ICA applied")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 13 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [85]:
# Save file
file_name = session_id+'_'+posner_id+'.vhdr'
output_dir='cleaned'

print(file_name)
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir, session_id)
print(session_dir)

if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name) 
mne.export.export_raw(session_dir+'/'+file_name, reconstr_raw, verbose=1, overwrite=True)

m_00_01_pos1a.vhdr
/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_00_01

**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_00_01/m_00_01_pos1a.vhdr


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [86]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_00_01
├── m_00_01_pos1a.eeg
├── m_00_01_pos1a.vhdr
└── m_00_01_pos1a.vmrk

1 directory, 3 files


In [87]:
del raw_a
del reconstr_raw
del ica
del filt_raw_a

In [52]:
### TESTING ###
# test = mne.io.read_raw_brainvision(session_dir+'/'+file_name, preload=True)

In [53]:
# test.plot()

# Posner B

In [88]:
# Relabel stim events and load raw data
raw_b = relabel_events(session_id, posner_id_b)

Data file:  m_00_01/m_00_01_pos1b.vhdr


/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Found '1a' in file: m_00_01/00_1a_posner_task_2023-04-05_15h00.59.401/00_1a_posner_task_2023-04-05_15h00.59.401.csv
Found '1b' in file: m_00_01/00_1b_posner_task_2023-04-05_16h08.54.140/00_1b_posner_task_2023-04-05_16h08.54.140.csv
********** 1b
Used Annotations descriptions: ['New Segment/', 'Stimulus/S 10', 'Stimulus/S 20', 'Stimulus/S 30', 'Stimulus/S 40', 'Stimulus/S 50', 'Stimulus/S 51', 'Stimulus/S 60', 'Stimulus/S 70', 'Stimulus/S 80', 'Stimulus/S101', 'Stimulus/S109', 'Stimulus/S117']
Cropping annotations 2023-04-05 16:09:44.710965+00:00 - 2023-04-05 16:23:37.550965+00:00
  [0] Keeping  (2023-04-05 16:09:44.710965+00:00 - 2023-04-05 16:09:44.710965+00:00 -> 0.0 - 0.0)
  [1] Keeping  (2023-04-05 16:11:24.308965+00:00 - 2023-04-05 1

/Users/babe/src/gla/dissertation/go/ica_notebooks/rename_events.py:50: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  raw = mne.io.read_raw_brainvision(vhdr_file, preload=True, verbose=0)


  [360] Keeping  (2023-04-05 16:21:24.253965+00:00 - 2023-04-05 16:21:24.253965+00:00 -> 699.543 - 699.543)
  [361] Keeping  (2023-04-05 16:21:26.873965+00:00 - 2023-04-05 16:21:26.873965+00:00 -> 702.163 - 702.163)
  [362] Keeping  (2023-04-05 16:21:26.973965+00:00 - 2023-04-05 16:21:26.973965+00:00 -> 702.263 - 702.263)
  [363] Keeping  (2023-04-05 16:21:29.254965+00:00 - 2023-04-05 16:21:29.254965+00:00 -> 704.544 - 704.544)
  [364] Keeping  (2023-04-05 16:21:30.753965+00:00 - 2023-04-05 16:21:30.753965+00:00 -> 706.043 - 706.043)
  [365] Keeping  (2023-04-05 16:21:33.599965+00:00 - 2023-04-05 16:21:33.599965+00:00 -> 708.889 - 708.889)
  [366] Keeping  (2023-04-05 16:21:33.703965+00:00 - 2023-04-05 16:21:33.703965+00:00 -> 708.993 - 708.993)
  [367] Keeping  (2023-04-05 16:21:35.763965+00:00 - 2023-04-05 16:21:35.763965+00:00 -> 711.053 - 711.053)
  [368] Keeping  (2023-04-05 16:21:37.264965+00:00 - 2023-04-05 16:21:37.264965+00:00 -> 712.554 - 712.554)
  [369] Keeping  (2023-04-05

In [89]:
event_map = {'1': 11, '10': 10, '101': 101, '109': 109, '11': 11, '117': 117, '20': 20, '30': 30, '40': 40, '5': 5, '50': 50, '51': 51, '52': 52, '60': 60, '70': 70, '80': 80, '99999': 99999}

# get events and times
events_b, event_id_b = mne.events_from_annotations(raw_b, event_map)

e_b = events_b
# all rows with fixation (code 40)
e_40_b = e_b[e_b[:, 2] == 40]
# all rows with trial end (code 101)
e_101_b = e_b[e_b[:,2]==101]

b_tmin= e_40_b[0][0]/1000
b_tmax= e_101_b[-1][0]/1000

# Crop and make a copy and filter
filt_raw_b = raw_b.crop(tmin=b_tmin, tmax=b_tmax).copy().filter(l_freq=1.0, h_freq=40)

epochs_b = mne.Epochs(filt_raw_b, 
                    e_b, 
                    event_id=None, 
                    tmin=-0.2, 
                    tmax=0.5, 
                    preload=True, 
                    baseline=(None, 0))

Used Annotations descriptions: ['10', '101', '109', '117', '20', '30', '40', '50', '51', '52', '60', '70', '80', '99999']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Not setting metadata
415 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 415 events and 701 original time points ...
6 bad epochs dropped


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.7s finished


In [5]:
raw_b.plot()

Using qt as 2D backend.
Using pyopengl with version 3.1.6


Channels marked as bad:
none


In [90]:
# Fitting ICA
ica_b = ICA(n_components=len(filt_raw_b.ch_names)-1, max_iter="auto", random_state=999)
ica_b.fit(filt_raw_b, verbose='DEBUG')
ica_b

Fitting ICA to data using 63 channels (please be patient, this may take a while)
Selecting by number: 63 components


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Cropping annotations 2023-04-05 16:11:31.870965+00:00 - 2023-04-05 16:22:52.180965+00:00
  [0] Keeping  (2023-04-05 16:11:31.870965+00:00 - 2023-04-05 16:11:31.870965+00:00 -> 107.16 - 107.16)
  [1] Keeping  (2023-04-05 16:11:33.370965+00:00 - 2023-04-05 16:11:33.370965+00:00 -> 108.66 - 108.66)
  [2] Keeping  (2023-04-05 16:11:36.550965+00:00 - 2023-04-05 16:11:36.550965+00:00 -> 111.84 - 111.84)
  [3] Keeping  (2023-04-05 16:11:36.650965+00:00 - 2023-04-05 16:11:36.650965+00:00 -> 111.94 - 111.94)
  [4] Keeping  (2023-04-05 16:11:38.377965+00:00 - 2023-04-05 16:11:38.377965+00:00 -> 113.667 - 113.667)
  [5] Keeping  (2023-04-05 16:11:39.880965+00:00 - 2023-04-05 16:11:39.880965+00:00 -> 115.17 - 115.17)
  [6] Keeping  (2023-04-05 16:11:43.520965+00:00 - 2023-04-05 16:11:43.520965+00:00 -> 118.81 - 118.81)
  [7] Keeping  (2023-04-05 16:11:43.620965+00:00 - 2023-04-05 16:11:43.620965+00:00 -> 118.91 - 118.91)
  [8] Keeping  (2023-04-05 16:11:44.880965+00:00 - 2023-04-05 16:11:44.880965

Method,fastica
Fit,1000 iterations on raw data (680310 samples)
ICA components,63
Available PCA components,63
Channel types,eeg
ICA components marked for exclusion,—


In [91]:
ica_b.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 585x260 with 3 Axes>]

In [ ]:
raw_b.load_data()
ica_b.plot_sources(raw_b)

Creating RawArray with float64 data, n_channels=63, n_times=680310
    Range : 107160 ... 787469 =    107.160 ...   787.469 secs
Ready.
Using pyopengl with version 3.1.6


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items activated
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
340 matching events found
No baseline correction applied
0 projection items ac

In [92]:
ica_b.exclude

[53, 54, 44, 21, 24, 1, 0, 2, 4, 3, 11, 14]

In [93]:
reconstr_raw_b = raw_b.copy()
# Apply ica to copy
ica_b.apply(reconstr_raw_b)
# Plot and compare
raw_b.plot(title="Raw data (posner b)")
reconstr_raw_b.plot(title="ICA applied (posner b)")

Applying ICA to Raw instance
    Transforming to ICA space (63 components)
    Zeroing out 12 ICA components
    Projecting back using 63 PCA components
Using pyopengl with version 3.1.6
Using pyopengl with version 3.1.6


Channels marked as bad:
none
Channels marked as bad:
none


In [96]:
# Save file
file_name_b = session_id+'_'+posner_id_b+'.vhdr'
output_dir_b='cleaned'
cur_dir = os.getcwd()
session_dir = os.path.join(cur_dir, output_dir_b, session_id)
if not os.path.exists(session_dir):
   os.makedirs(session_dir)

print("\n**** Saving file to: ", session_dir+'/'+file_name_b) 
mne.export.export_raw(session_dir+'/'+file_name_b, reconstr_raw_b, verbose=1, overwrite=True)


**** Saving file to:  /Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_00_01/m_00_01_pos1b.vhdr
Overwriting existing file.


/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/_export.py:61: RuntimeWarning: Encountered data in 'short' format. Converting to float32.
  warn(
/Applications/MNE-Python/.mne-python/lib/python3.10/site-packages/pybv/io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


In [97]:
!tree {session_dir}

/Users/babe/src/gla/dissertation/go/ica_notebooks/cleaned/m_00_01
├── m_00_01_pos1a.eeg
├── m_00_01_pos1a.vhdr
├── m_00_01_pos1a.vmrk
├── m_00_01_pos1b.eeg
├── m_00_01_pos1b.vhdr
└── m_00_01_pos1b.vmrk

1 directory, 6 files


# ...

In [98]:
del raw_b
del reconstr_raw_b
del ica_b
del filt_raw_b